# Feature Store Query Planner Tutorial

The query planner in the Hopsworks Feature Store allows to query the feature store with high level functions, such as `get_features`, and `get_featuregroup`. However, it is only intended as a utility tool, and the user can always fall back to using pure SQL to query the feature store.

In this notebook we will go over a common "gotcha" when using the featurestore query planner: **naming conflicts**. A feature name is only guaranteed to be unique within its feature group and version. The feature name **is not globally unique**; this means that conflicts can occur. In this notebook we will illustrate how to handle querying the feature store when there are conflicts.

TLDR;

If there is a conflict, you can fall back to using pure SQL to query the feature store, or you can provide extra information to the query planner in order to resolve the conflict.

## Inspect two Feature Groups with a Naming Conflict on Column: `team_id`

In this example we want to make a feature query that cross two feature groups, that have a common column `team_id` that can be used for joining features together.

In [ ]:
from hops import featurestore

In [3]:
featurestore.get_featuregroup("attendances_features").printSchema()

Running sql: use demo_featurestore_admin000_featurestore against offline feature store
SQL string for the query created successfully
Running sql: SELECT * FROM attendances_features_1 against offline feature store
root
 |-- team_id: integer (nullable = true)
 |-- average_attendance: float (nullable = true)
 |-- sum_attendance: float (nullable = true)

In [4]:
featurestore.get_featuregroup("teams_features").printSchema()

Running sql: use demo_featurestore_admin000_featurestore against offline feature store
SQL string for the query created successfully
Running sql: SELECT * FROM teams_features_1 against offline feature store
root
 |-- team_budget: float (nullable = true)
 |-- team_id: integer (nullable = true)
 |-- team_position: integer (nullable = true)

## Fetch a list of features from two feature groups

### If there is not any naming conflict, the Query Planner can figure out how to fetch the features and join them

In [23]:
featurestore.get_features(["average_attendance", "sum_attendance", "team_budget", "team_position"]).printSchema()

Running sql: use demo_featurestore_admin000_featurestore against offline feature store
Logical query plan for getting 4 features from the featurestore created successfully
SQL string for the query created successfully
Running sql: SELECT sum_attendance, team_budget, average_attendance, team_position FROM attendances_features_1 JOIN teams_features_1 ON attendances_features_1.`team_id`=teams_features_1.`team_id` against offline feature store
root
 |-- sum_attendance: float (nullable = true)
 |-- team_budget: float (nullable = true)
 |-- average_attendance: float (nullable = true)
 |-- team_position: integer (nullable = true)

### The Query Planner is only intended as a utility, you can always fall back to pure SQL to query the feature store, which gives you more control

In [26]:
spark.sql("SELECT sum_attendance, team_budget, average_attendance, team_position " \
          "FROM attendances_features_1 JOIN teams_features_1 ON attendances_features_1.`team_id`=teams_features_1.`team_id`").printSchema()

root
 |-- sum_attendance: float (nullable = true)
 |-- team_budget: float (nullable = true)
 |-- average_attendance: float (nullable = true)
 |-- team_position: integer (nullable = true)

### If there is a naming conflict, the query planner will ask us to specify the feature groups to fetch the features from

In [24]:
featurestore.get_features(["average_attendance", "sum_attendance", "team_budget", "team_position", "team_id"]).printSchema()

Found the feature with name 'team_id' in more than one of the featuregroups of the featurestore: 'demo_featurestore_admin000_featurestore', please specify the optional argument 'featuregroup=', the matched featuregroups were: season_scores_features_1,attendances_features_1,players_features_1,teams_features_1
Traceback (most recent call last):
  File "/srv/hops/anaconda/anaconda/envs/python36/lib/python3.6/site-packages/hops/featurestore.py", line 367, in get_features
    online=online)
  File "/srv/hops/anaconda/anaconda/envs/python36/lib/python3.6/site-packages/hops/featurestore_impl/core.py", line 611, in _do_get_features
    logical_query_plan.create_logical_plan()
  File "/srv/hops/anaconda/anaconda/envs/python36/lib/python3.6/site-packages/hops/featurestore_impl/query_planner/logical_query_plan.py", line 32, in create_logical_plan
    self._features_query()
  File "/srv/hops/anaconda/anaconda/envs/python36/lib/python3.6/site-packages/hops/featurestore_impl/query_planner/logical_qu

### If there is a naming conflict even in the list of feature groups provided, SparkSQL will complain that the conflicting feature must be specified with the full name `tablename_featurename`.

By specifying the list of feature groups and their version, we are effectively pruning the query planner to only look inside these feature groups. This reduces the chance of conflicts, but it is not a guaranteed protection against naming conflicts: there can be conflicts inside the selected feature groups as well.

In [25]:
featurestore.get_features(["average_attendance", "sum_attendance", "team_budget", "team_position", "team_id"], 
                          featuregroups_version_dict = {"attendances_features": 1, "teams_features": 1}).printSchema()

"Reference 'team_id' is ambiguous, could be: demo_featurestore_admin000_featurestore.attendances_features_1.team_id, demo_featurestore_admin000_featurestore.teams_features_1.team_id.; line 1 pos 36"
Traceback (most recent call last):
  File "/srv/hops/anaconda/anaconda/envs/python36/lib/python3.6/site-packages/hops/featurestore.py", line 367, in get_features
    online=online)
  File "/srv/hops/anaconda/anaconda/envs/python36/lib/python3.6/site-packages/hops/featurestore_impl/core.py", line 617, in _do_get_features
    result = _run_and_log_sql(spark, logical_query_plan.sql_str, online=online, featurestore=featurestore)
  File "/srv/hops/anaconda/anaconda/envs/python36/lib/python3.6/site-packages/hops/featurestore_impl/core.py", line 424, in _run_and_log_sql
    return spark.sql(sql_str)
  File "/srv/hops/spark/python/lib/pyspark.zip/pyspark/sql/session.py", line 767, in sql
    return DataFrame(self._jsparkSession.sql(sqlQuery), self._wrapped)
  File "/srv/hops/spark/python/lib/py4j-s

### Same Error Occurs if we Skip the Query Planner and Query Hive Directly to get the list of features, as it is not valid SparkSQL

In [20]:
spark.sql("SELECT team_budget, sum_attendance, team_id, team_position, average_attendance " \
          "FROM attendances_features_1 JOIN teams_features_1 ON attendances_features_1.`team_id`=teams_features_1.`team_id`").printSchema()

"Reference 'team_id' is ambiguous, could be: demo_featurestore_admin000_featurestore.attendances_features_1.team_id, demo_featurestore_admin000_featurestore.teams_features_1.team_id.; line 1 pos 36"
Traceback (most recent call last):
  File "/srv/hops/spark/python/lib/pyspark.zip/pyspark/sql/session.py", line 767, in sql
    return DataFrame(self._jsparkSession.sql(sqlQuery), self._wrapped)
  File "/srv/hops/spark/python/lib/py4j-src.zip/py4j/java_gateway.py", line 1257, in __call__
    answer, self.gateway_client, self.target_id, self.name)
  File "/srv/hops/spark/python/lib/pyspark.zip/pyspark/sql/utils.py", line 69, in deco
    raise AnalysisException(s.split(': ', 1)[1], stackTrace)
pyspark.sql.utils.AnalysisException: "Reference 'team_id' is ambiguous, could be: demo_featurestore_admin000_featurestore.attendances_features_1.team_id, demo_featurestore_admin000_featurestore.teams_features_1.team_id.; line 1 pos 36"



### To avoid the  error we can specify the conflicting feature name with a full name (`featuregroupname`_`version`.`featurename`)

In [27]:
featurestore.get_features(["average_attendance", "sum_attendance", "team_budget", "team_position", "attendances_features_1.team_id"], 
                          featuregroups_version_dict = {"attendances_features": 1, "teams_features": 1}).printSchema()

Running sql: use demo_featurestore_admin000_featurestore against offline feature store
Logical query plan for getting 5 features from the featurestore created successfully
SQL string for the query created successfully
Running sql: SELECT team_budget, sum_attendance, attendances_features_1.team_id, team_position, average_attendance FROM attendances_features_1 JOIN teams_features_1 ON attendances_features_1.`team_id`=teams_features_1.`team_id` against offline feature store
root
 |-- team_budget: float (nullable = true)
 |-- sum_attendance: float (nullable = true)
 |-- team_id: integer (nullable = true)
 |-- team_position: integer (nullable = true)
 |-- average_attendance: float (nullable = true)

In [11]:
featurestore.get_features(["average_attendance", "sum_attendance", "team_budget", "team_position", "attendances_features_1.team_id", 
                           "teams_features_1.team_id"], 
                          featuregroups_version_dict = {"attendances_features": 1, "teams_features": 1}).printSchema()

Running sql: use demo_featurestore_admin000_featurestore against offline feature store
Logical query plan for getting 5 features from the featurestore created successfully
SQL string for the query created successfully
Running sql: SELECT team_budget, sum_attendance, attendances_features_1.team_id, team_position, average_attendance FROM attendances_features_1 JOIN teams_features_1 ON attendances_features_1.`team_id`=teams_features_1.`team_id` against offline feature store
root
 |-- team_budget: float (nullable = true)
 |-- sum_attendance: float (nullable = true)
 |-- team_id: integer (nullable = true)
 |-- team_position: integer (nullable = true)
 |-- average_attendance: float (nullable = true)

In [13]:
featurestore.get_features(["average_attendance", "sum_attendance", "team_budget", "team_position", "attendances_features_1.team_id"], 
                          featuregroups_version_dict = {"attendances_features": 1, "teams_features": 1}).printSchema()

Running sql: use demo_featurestore_admin000_featurestore against offline feature store
Logical query plan for getting 5 features from the featurestore created successfully
SQL string for the query created successfully
Running sql: SELECT team_budget, sum_attendance, attendances_features_1.team_id, team_position, average_attendance FROM attendances_features_1 JOIN teams_features_1 ON attendances_features_1.`team_id`=teams_features_1.`team_id` against offline feature store
root
 |-- team_budget: float (nullable = true)
 |-- sum_attendance: float (nullable = true)
 |-- team_id: integer (nullable = true)
 |-- team_position: integer (nullable = true)
 |-- average_attendance: float (nullable = true)

In [29]:
spark.sql("SELECT team_budget, sum_attendance, attendances_features_1.team_id, team_position, average_attendance " \
          "FROM attendances_features_1 JOIN teams_features_1 ON attendances_features_1.`team_id`=teams_features_1.`team_id`").printSchema()

root
 |-- team_budget: float (nullable = true)
 |-- sum_attendance: float (nullable = true)
 |-- team_id: integer (nullable = true)
 |-- team_position: integer (nullable = true)
 |-- average_attendance: float (nullable = true)